In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import csv
import pandas as pd
import numpy as np
# do the LDA
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
import nltk
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
with open('Feature_List', newline='') as f:
    reader = csv.reader(f)
    feature_list = list(reader)

In [8]:
BA_feature=feature_list[0]
DA_feature=feature_list[1]
DS_feature=feature_list[2]
DE_feature=feature_list[3]

In [9]:
all_feature=BA_feature+DA_feature+DS_feature+DE_feature

In [13]:
len(all_feature)

40

In [12]:
len(set(all_feature))

23

In [25]:
BA_JD=pd.read_csv("BA_JD.csv")[:5]
BA_JD

,Unnamed: 0,position,company,location,details
0,0,Business Analyst Intern,RM Talent Solutions,"Washington, United States",Posted by\nAzhar Ali Khan\nTalent Acquisition ...
1,1,Business Analyst Intern,RM Talent Solutions,"Maine, United States",Posted by\nKhalid Ansari\nRecruitment Speciali...
2,2,Business Analyst,Cisco,"New York, NY",Who We Are\n\nCisco Security – at Cisco we del...
3,3,U.S. Private Banking & Wealth Business Analyti...,Citi,"New York, NY",Job Id: 22461117\n\nThis is a Business Analyti...
4,4,Business Analyst Intern,2 a.m. Talent,"Jacksonville, FL",Responsibilities vary depending on assigned pr...


In [29]:
def JD_filter(df):
    
    l1=""
    for i in df.index:
        sents=df.loc[i]['details']
        l1+=(sents.replace('\n',''))
        word_tokens = word_tokenize(l1)
        stop_words = set(stopwords.words('english')) 
        wordlist=[word for word in word_tokens if word.isalnum() and word not in stop_words]
        full_text=""
        for word in wordlist:
            full_text+=word+" "
        df["details"][i]=full_text
    return df
    

In [52]:
BA=JD_filter(BA_JD)

<ipython-input-29-17542096419d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["details"][i]=full_text


In [47]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

ps = PorterStemmer()

# process the keywords
ba_feature_set = set([ps.stem(tok) for tok in BA_feature]) 
da_feature_set = set([ps.stem(tok) for tok in DA_feature]) 
ds_feature_set = set([ps.stem(tok) for tok in DS_feature]) 
de_feature_set = set([ps.stem(tok) for tok in DE_feature])
all_feature_set = set([ps.stem(tok) for tok in all_feature])

In [48]:
# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    desc=desc.replace("-"," ")
    desc=desc.replace("/"," ")
    tokens = [x for x in word_tokenize(desc) if x.isalnum()]
        
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok, tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(tok).lower() for tok in filtered_tokens]
    return set(stemmed_tokens)

In [49]:
def mapping_list(df):
    #name=get_df_name(df)
    #df['details']=name.split("_")[1]
    df['job_description_word_set'] =df['details'].map(prepare_job_desc)
    
    ba_feature = []
    da_feature = []
    ds_feature = []
    de_feature = []
    all_feature = []
   

    num_postings = len(df.index)
    for i in range(num_postings):
        job_desc = df.iloc[i]['details'].lower()
        job_desc_set = df.iloc[i]['job_description_word_set']

        # check if the keywords are in the job description. Look for exact match by token.
        ba = ba_feature_set.intersection(job_desc_set)
        da = da_feature_set.intersection(job_desc_set)
        ds = ds_feature_set.intersection(job_desc_set)
        de = de_feature_set.intersection(job_desc_set)
        al = all_feature_set.intersection(job_desc_set)

        
        if len(ba) == 0:
            ba=[None]
        if len(da) == 0:
            da=[None]
        if len(ds) == 0:
            ds=[None]
        if len(de) == 0:
            de=[None]
        if len(al) == 0:
            al=[None]
    
                
        ba_feature.append(list(ba))
        da_feature.append(list(da))
        ds_feature.append(list(ds))
        de_feature.append(list(de))
        all_feature.append(list(al))
    
    return ba_feature, da_feature, ds_feature, de_feature,all_feature

In [50]:
def df_with_feature(df):
    ba, da, ds, de, al= mapping_list(df)
    df["ba"]=ba
    df["da"]=da
    df["ds"]=ds
    df["de"]=ds
    df["all"]=al
   
    return df

In [51]:
ba_final=df_with_feature(BA_JD)
ba_final

,Unnamed: 0,position,company,location,details,job_description_word_set,ba,da,ds,de,all
0,0,Business Analyst Intern,RM Talent Solutions,"Washington, United States",Posted byAzhar Ali KhanTalent Acquisition Hiri...,"{problem, stakehold, top, author, applic, expl...","[data, commun, busi]","[data, commun, busi]","[data, techniqu, busi]","[data, techniqu, busi]","[data, techniqu, commun, busi]"
1,1,Business Analyst Intern,RM Talent Solutions,"Maine, United States",Posted byAzhar Ali KhanTalent Acquisition Hiri...,"{intellig, 2022, encourag, grow, problem, 500,...","[data, commun, busi]","[data, commun, busi]","[data, techniqu, busi]","[data, techniqu, busi]","[data, techniqu, commun, busi]"
2,2,Business Analyst,Cisco,"New York, NY",Posted byAzhar Ali KhanTalent Acquisition Hiri...,"{global, juggl, reduc, intellig, 2022, progres...","[engin, data, commun, busi]","[busi, data, commun, cloud]","[data, techniqu, busi]","[data, techniqu, busi]","[commun, techniqu, cloud, engin, data, busi]"
3,3,U.S. Private Banking & Wealth Business Analyti...,Citi,"New York, NY",Posted byAzhar Ali KhanTalent Acquisition Hiri...,"{intellig, sexual, grow, unconnect, idea, thre...","[commun, engin, data, statist, busi]","[commun, sql, cloud, learn, data, busi]","[techniqu, learn, data, statist, busi]","[techniqu, learn, data, statist, busi]","[commun, sql, learn, techniqu, cloud, engin, d..."
4,4,Business Analyst Intern,2 a.m. Talent,"Jacksonville, FL",Posted byAzhar Ali KhanTalent Acquisition Hiri...,"{intellig, sexual, grow, unconnect, idea, thre...","[commun, engin, data, statist, busi]","[commun, sql, cloud, learn, data, busi]","[techniqu, learn, data, statist, busi]","[techniqu, learn, data, statist, busi]","[commun, sql, learn, techniqu, cloud, engin, d..."
